In [1]:
import ads
import base64
import numpy as np
import tensorflow as tf

from ads.common.model_metadata import UseCaseType
from ads.model.framework.tensorflow_model import TensorFlowModel
from io import BytesIO
from uuid import uuid4

ads.set_auth(auth="resource_principal")

# Create the TensorFlow Model
mnist = tf.keras.datasets.mnist
(trainx, trainy), (testx, testy) = mnist.load_data()
trainx, testx = trainx / 255.0, testx / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(trainx, trainy, epochs=1)

In [3]:
# Exercise/validate the prediction
model.predict(testx[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


array([[-1.243576  , -6.0396724 ,  0.21659924,  3.1436307 , -6.4149337 ,
        -0.62004346, -9.847992  , 10.09782   , -1.149577  ,  0.9160745 ]],
      dtype=float32)

In [4]:
## TENSORFLOW FRAMEWORK SERIALIZATION
# Create an ADS model
tensorflow_model = TensorFlowModel(estimator=model, artifact_dir=f"./model-artifact-{str(uuid4())}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [5]:
# Prepare the ADS Model
tensorflow_model.prepare(
    X_sample=trainx,
    y_sample=trainy,
    use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION,
)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [6]:
# Verify the ADS model
tensorflow_model.verify(testx)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-d26fb701-81fa-41bf-8186-81d72aedd586 ...
Model is successfully loaded.


Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [7]:
# Save the ADS model
model_id = tensorflow_model.save()

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-d26fb701-81fa-41bf-8186-81d72aedd586 ...
Model is successfully loaded.
['output_schema.json', 'score.py', '.model-ignore', 'model.h5', 'runtime.yaml']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [8]:
# Deploy and create an endpoint for the TensorFlow model
tensorflow_model.deploy(
    display_name="DIY TensorFlow Model For MNIST Classification",
    #deployment_log_group_id="ocid1.loggroup.oc1.xxx.xxxxx",
    #deployment_access_log_id="ocid1.log.oc1.xxx.xxxxx",
    #deployment_predict_log_id="ocid1.log.oc1.xxx.xxxxx",
    # Shape config details mandatory for flexible shapes:
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=1,
    deployment_memory_in_gbs=16,
)
print(f"Endpoint: {tensorflow_model.model_deployment.url}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Model Deployment OCID: ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiaqvt5gjeipsxfxcan4swhc5kuvqt6orhwuenkca2n2soa


Creating model deployment:   0%|          | [00:00<?, ?it/s]

ERROR:ads.common.work_request:Exceeded max wait time of 1200 seconds.
Endpoint: https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiaqvt5gjeipsxfxcan4swhc5kuvqt6orhwuenkca2n2soa


Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  CREATING      Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [9]:
# Generate prediction by invoking the deployed endpoint
tensorflow_model.predict(testx[:3])['prediction']

ERROR - Exception
Traceback (most recent call last):
  File "/home/datascience/conda/onac_diy_v2_0/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_30195/838254498.py", line 2, in <module>
    tensorflow_model.predict(testx[:3])['prediction']
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/datascience/conda/onac_diy_v2_0/lib/python3.11/site-packages/ads/model/generic_model.py", line 3516, in predict
    return super().predict(
           ^^^^^^^^^^^^^^^^
  File "/home/datascience/conda/onac_diy_v2_0/lib/python3.11/site-packages/ads/model/generic_model.py", line 2940, in predict
    raise NotActiveDeploymentError(current_state)
ads.model.generic_model.NotActiveDeploymentError: To perform a prediction the deployed model needs to be in an active state. The current state is: CREATING.
NotActiveDeploymentError: To perform a prediction the deployed model needs to be in an 

In [ ]:
# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

In [ ]:
# Prepare data sample for prediction and save it to file 'data-payload'
data = testx[:3]
np_bytes = BytesIO()
np.save(np_bytes, data, allow_pickle=True)
data = base64.b64encode(np_bytes.getvalue()).decode("utf-8")
with open('data-payload', 'w') as f:
    f.write('{"data": "' + data + '", "data_type": "numpy.ndarray"}')

In [ ]:
# Alternately, the OCI CLI may be invoked with the command line
print("Use the following commands in the ODSC terminal to invoke the deployment via the OCI CLI...\n")
print("$ cd /home/datascience/demo")
print("$ oci raw-request --http-method POST --target-uri " + tensorflow_model.model_deployment.url + "/predict --request-body file://data-payload")